In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import RandomNormal

WTInit = RandomNormal(mean=0.0, stddev=0.01, seed=5)
model=Sequential()
model.add(Convolution2D(16, (5, 5), input_shape=(64, 64, 6), activation="relu", kernel_initializer=WTInit, bias_initializer="zeros"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (5, 5), activation="relu", kernel_initializer=WTInit, bias_initializer="zeros"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (5, 5), activation="relu", kernel_initializer=WTInit, bias_initializer="zeros"))

model.add(Flatten())

model.add(Dense(640, activation="relu", kernel_initializer=WTInit, bias_initializer="zeros"))
model.add(Dense(2, activation="softmax", kernel_initializer=WTInit, bias_initializer="zeros"))

sgd = SGD(lr= 0.01, momentum=0.9, decay=0.005)
model.compile(optimizer=sgd, loss="categorical_crossentropy")
print(model.summary())

C:\Users\Simran\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        25664     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         204928    
_________________________________________________________________
flatten_1 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 640)               6636160   
__________

In [2]:
import numpy as np
import imageio
import scipy.io
import pandas as pd

#def LoadData(DS='KinFaceW-I'):
#mat_fd = scipy.io.loadmat(DS + '/fd_pairs.mat')
#s = DS + '/images/'+ string[m]
#s =  '/images/'+ string[m]
def LoadData():
	count=0
	all_images=[]
	Kin = pd.DataFrame(columns=['Fold', 'Kin/Not-Kin'])

	mat_fd = scipy.io.loadmat( 'fd_pairs_2.mat')
	mat_fd = mat_fd["pairs"]
	mat_fs = scipy.io.loadmat('fs_pairs_2.mat')
	mat_fs = mat_fs["pairs"]
	mat_md = scipy.io.loadmat('md_pairs_2.mat')
	mat_md = mat_md["pairs"]
	mat_ms = scipy.io.loadmat('ms_pairs_2.mat')
	mat_ms = mat_ms["pairs"]

	Mat = [mat_fd, mat_fs, mat_md, mat_ms]
	string = ['father-dau_2/fd_', 'father-son_2/fs_', 'mother-dau_2/md_', 'mother-son_2/ms_']

	for m in range(0, 4):
		for j in range(0, Mat[m].shape[0]):
			s =  string[m]
			addr = s + Mat[m][j][2][0][3:6]
			image1 = imageio.imread(addr +'_1.jpg')	
			addr = s + Mat[m][j][3][0][3:6]  
			image2 = imageio.imread(addr +'_2.jpg')
			Kin.loc[count] = [Mat[m][j][0][0][0], Mat[m][j][1][0][0]]
			new_image = np.concatenate((image1, image2), axis=2)
			all_images+=[np.array(new_image)]
			count+=1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
	all_images = np.array(all_images)
	all_images = all_images.astype('float32')
	all_images -= np.mean(all_images, axis=0)
	all_images /= np.std(all_images, axis=0)
	Kin = np.array(Kin)
	Data = [all_images, Kin]

	rng_state = np.random.get_state()
	np.random.shuffle(all_images)
	np.random.set_state(rng_state)
	np.random.shuffle(Kin) 

	Folds = [[[], []], [[], []], [[], []], [[], []], [[], []]]
	for i in range (0, all_images.shape[0]):
		for j in range(0, 5):
			if(Data[1][i][0]==j+1):
				Folds[j] = np.append(Folds[j], [[np.array(all_images[i])], [np.array(Kin[i])]], axis=1)

	X=[[], [], [], [], []]
	Y=[[], [], [], [], []] 
	for i in range(0, 5): 
		X[i] = np.array(Folds[i][0])
		Y[i] = np.array([Folds[i][1][j][1] for j in range(Folds[i].shape[1])]) 

	X_Train=[[], [], [], [], []] 
	Y_Train=[[], [], [], [], []]    
	X_Test=[[], [], [], [], []]
	Y_Test=[[], [], [], [], []]    
    
	for i in range(0, 5):    
		X_Train[i] = np.append(X[i%5], X[(i+1)%5], axis=0)
		X_Train[i] = np.append(X_Train[i], X[(i+2)%5], axis=0)
		X_Train[i] = np.append(X_Train[i], X[(i+3)%5], axis=0)
		Y_Train[i] = np.append(Y[i%5], Y[(i+1)%5], axis=0)
		Y_Train[i] = np.append(Y_Train[i], Y[(i+2)%5], axis=0)
		Y_Train[i] = np.append(Y_Train[i], Y[(i+3)%5], axis=0)
		X_Test[i] = X[(i+4)%5]
		Y_Test[i] = Y[(i+4)%5]

	return X_Train, Y_Train, X_Test, Y_Test	

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import RandomNormal
from keras.layers import BatchNormalization

def CompileModel():
	model=Sequential()
	model.add(Convolution2D(16, (5, 5), input_shape=(64, 64, 6)))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Convolution2D(64, (5, 5)))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	model.add(Convolution2D(256, (5, 5)))
	model.add(BatchNormalization())
	model.add(Activation('relu'))

	model.add(Flatten())
	model.add(Dropout(0.5))
    
	model.add(Dense(640))
	model.add(BatchNormalization())
	model.add(Activation('relu'))

	model.add(Dropout(0.5))
	model.add(Dense(2))
	model.add(Activation('softmax'))

	sgd = SGD(lr=0.0001, momentum=0.9, decay=0.005)
	model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=['accuracy'])
	print(model.summary())
	return model

In [4]:
import numpy as np
import keras
import sys

sys.path.append('/home/inertfluid/.config/spyder-py3/')

#from DataPreProcess import LoadData
#from ModelCompiler import CompileModel

DataSet = 'KinFaceW-II'
model = CompileModel()
X_Train, Y_Train, X_Test, Y_Test = LoadData()

score=[]
for j in range(0, 5):
    X_Train[j] = np.reshape(list(X_Train[j]), (X_Train[j].shape[0], 64, 64, 6))
    Y_Train[j] = keras.utils.to_categorical(Y_Train[j], 2)
    X_Test[j] = np.reshape(list(X_Test[j]), (X_Test[j].shape[0], 64, 64, 6))
    Y_Test[j] = keras.utils.to_categorical(Y_Test[j], 2)
    model.fit(X_Train[j], Y_Train[j], batch_size=64, epochs=50, validation_data=(X_Test[j], Y_Test[j]), shuffle=True)
    score+=[model.evaluate(X_Test[j], Y_Test[j], verbose=1)]
    
score=np.array(score)
loss = np.average(score.T[0])
accuracy = np.average(score.T[1])

print('Loss:', loss)
print('Accuracy:', accuracy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 60, 60, 16)        2416      
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 64)        25664     
_________________________________________________________________
batch_normalization_2 (Batch (None, 26, 26, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 26, 26, 64)        0         
__________

1600/1600 [==============================] - 92s 58ms/step - loss: 0.0662 - acc: 0.9763 - val_loss: 0.3725 - val_acc: 0.8575
Epoch 40/50
1600/1600 [==============================] - 99s 62ms/step - loss: 0.0770 - acc: 0.9756 - val_loss: 0.3804 - val_acc: 0.8475
Epoch 41/50
1600/1600 [==============================] - 112s 70ms/step - loss: 0.0710 - acc: 0.9731 - val_loss: 0.3988 - val_acc: 0.8625
Epoch 42/50
1600/1600 [==============================] - 107s 67ms/step - loss: 0.0716 - acc: 0.9731 - val_loss: 0.4081 - val_acc: 0.8625
Epoch 43/50
1600/1600 [==============================] - 105s 66ms/step - loss: 0.0659 - acc: 0.9781 - val_loss: 0.3603 - val_acc: 0.8700
Epoch 44/50
1600/1600 [==============================] - 113s 71ms/step - loss: 0.0595 - acc: 0.9794 - val_loss: 0.3812 - val_acc: 0.8475
Epoch 45/50
1600/1600 [==============================] - 109s 68ms/step - loss: 0.0489 - acc: 0.9838 - val_loss: 0.4450 - val_acc: 0.8375
Epoch 46/50
1600/1600 [=========================

1600/1600 [==============================] - 96s 60ms/step - loss: 0.0211 - acc: 0.9925 - val_loss: 0.0268 - val_acc: 0.9925
Epoch 49/50
1600/1600 [==============================] - 95s 59ms/step - loss: 0.0258 - acc: 0.9913 - val_loss: 0.0258 - val_acc: 0.9925
Epoch 50/50
400/400 [==============================] - 10s 25ms/step
Train on 1600 samples, validate on 400 samples
Epoch 1/50
1600/1600 [==============================] - 96s 60ms/step - loss: 0.0363 - acc: 0.9881 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 2/50
1600/1600 [==============================] - 96s 60ms/step - loss: 0.0422 - acc: 0.9812 - val_loss: 0.0035 - val_acc: 1.0000
Epoch 3/50
1600/1600 [==============================] - 97s 60ms/step - loss: 0.0264 - acc: 0.9925 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 4/50
1600/1600 [==============================] - 98s 61ms/step - loss: 0.0414 - acc: 0.9881 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 5/50
1600/1600 [==============================] - 97s 61ms/step - loss: 

Epoch 6/50
1600/1600 [==============================] - 124s 77ms/step - loss: 0.0171 - acc: 0.9937 - val_loss: 2.4509e-04 - val_acc: 1.0000
Epoch 7/50
1600/1600 [==============================] - 124s 77ms/step - loss: 0.0147 - acc: 0.9975 - val_loss: 7.0704e-04 - val_acc: 1.0000
Epoch 8/50
1600/1600 [==============================] - 124s 77ms/step - loss: 0.0183 - acc: 0.9937 - val_loss: 3.9439e-04 - val_acc: 1.0000
Epoch 9/50
1600/1600 [==============================] - 124s 77ms/step - loss: 0.0126 - acc: 0.9956 - val_loss: 5.1913e-04 - val_acc: 1.0000
Epoch 10/50
1600/1600 [==============================] - 124s 77ms/step - loss: 0.0187 - acc: 0.9906 - val_loss: 3.2643e-04 - val_acc: 1.0000
Epoch 11/50
1600/1600 [==============================] - 126s 78ms/step - loss: 0.0232 - acc: 0.9931 - val_loss: 4.3817e-04 - val_acc: 1.0000
Epoch 12/50
1600/1600 [==============================] - 107s 67ms/step - loss: 0.0107 - acc: 0.9975 - val_loss: 5.9254e-04 - val_acc: 1.0000
Epoch 13/5

1600/1600 [==============================] - 92s 58ms/step - loss: 0.0126 - acc: 0.9956 - val_loss: 1.2746e-04 - val_acc: 1.0000
Epoch 14/50
1600/1600 [==============================] - 94s 59ms/step - loss: 0.0177 - acc: 0.9944 - val_loss: 2.6195e-04 - val_acc: 1.0000
Epoch 15/50
1600/1600 [==============================] - 93s 58ms/step - loss: 0.0076 - acc: 0.9981 - val_loss: 2.0502e-04 - val_acc: 1.0000
Epoch 16/50
1600/1600 [==============================] - 93s 58ms/step - loss: 0.0062 - acc: 0.9988 - val_loss: 1.7381e-04 - val_acc: 1.0000
Epoch 17/50
1600/1600 [==============================] - 91s 57ms/step - loss: 0.0076 - acc: 0.9981 - val_loss: 1.2117e-04 - val_acc: 1.0000
Epoch 18/50
1600/1600 [==============================] - 90s 57ms/step - loss: 0.0133 - acc: 0.9950 - val_loss: 8.5781e-04 - val_acc: 1.0000
Epoch 19/50
1600/1600 [==============================] - 91s 57ms/step - loss: 0.0110 - acc: 0.9969 - val_loss: 1.9614e-04 - val_acc: 1.0000
Epoch 20/50
1600/1600 [==